<a href="https://colab.research.google.com/github/sushant822/big-data-challenge/blob/main/level-2/big_data_challenge_video_games_l2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.7'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/u

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-01 23:26:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.06MB/s    in 0.2s    

2020-11-01 23:26:26 (5.06 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://classdb27oct2020.s3-us-west-1.amazonaws.com/amazon_reviews_us_Digital_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
digital_video_games_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv"), 
                                        sep="\t", header=True, inferSchema=True)

# Show DataFrame
digital_video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-08-31 00:00:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [5]:
selected_df = digital_video_games_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
selected_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          2|            2|          3|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          1|            0|          0|   N|                N|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|           

In [6]:
selected_df_votes = selected_df.filter(selected_df["total_votes"] >= 10)
selected_df_votes_filter = selected_df_votes.filter(selected_df_votes["helpful_votes"]/selected_df_votes["total_votes"] >= 0.5)

In [7]:
from pyspark.sql.functions import col, avg
verified_purchase_yes = selected_df_votes_filter.filter(selected_df_votes_filter['verified_purchase'] == 'Y')
verified_purchase_no = selected_df_votes_filter.filter(selected_df_votes_filter['verified_purchase'] == 'N')

verified_purchase_yes.describe().show()
verified_purchase_no.describe().show()

+-------+------------------+-----------------+------------------+----+-----------------+
|summary|       star_rating|    helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+-----------------+------------------+----+-----------------+
|  count|              2350|             2350|              2350|2350|             2350|
|   mean|3.1485106382978723|24.92595744680851|29.420851063829787|null|             null|
| stddev|1.7105549411142797|123.5619797604185|132.44370319532376|null|             null|
|    min|                 1|                5|                10|   N|                Y|
|    max|                 5|             5068|              5251|   N|                Y|
+-------+------------------+-----------------+------------------+----+-----------------+

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+---------

In [8]:
ratings_4_v = verified_purchase_yes[verified_purchase_yes['star_rating'] >= 4].count()
ratings_4_v_count = verified_purchase_yes.count()
ratings_v_percentage = round((float(ratings_4_v) / float(ratings_4_v_count))*100, 2)
print(f'Total number of ratings (4 or greater): {ratings_4_v_count}')
print(f'Total number of verified purchases: {ratings_4_v}')
print(f'Percentage: {ratings_v_percentage}%')

Total number of ratings (4 or greater): 2350
Total number of verified purchases: 1189
Percentage: 50.6%


In [ ]:
ratings_4_nv = verified_purchase_no[verified_purchase_no['star_rating'] >= 4].count()
ratings_4_nv_count = verified_purchase_no.count()
ratings_nv_percentage = round((float(ratings_4_nv) / float(ratings_4_nv_count))*100, 2)
print(f'Total number of ratings (4 or greater): {ratings_4_nv_count}')
print(f'Total number of non-verified purchases: {ratings_4_nv}')
print(f'Percentage: {ratings_nv_percentage}%')